In [151]:
import pandas as pd
import numpy as np
import requests,os,json
import time

from groq import Groq
from google.colab import userdata
API_KEY=userdata.get('ai_api_key')

client = Groq(

    api_key=API_KEY,

)

In [13]:
with open("/content/fortune_100_companies_json.txt", "r") as f:
        company_data = json.load(f)

In [51]:
company_data[9]['data']['Employees']

'44,000'

In [59]:
company_name=[]
industry=[]
hq=[]
annual_revenue=[]
employee_data=[]
for i in range (100):
  company_name.append(company_data[i]['name'])
  industry.append(company_data[i]['data']['Industry'])
  hq.append(company_data[i]['data']['Headquarters City']+", "+company_data[i]['data']['State'])
  annual_revenue.append(company_data[i]['data']['Revenues ($M)'])
  employee_data.append(company_data[i]['data']['Employees'])

In [60]:
dict = {'Company Name': company_name, 'Industry': industry, 'Headquarters Location':hq, 'Estimated Annual Revenue':annual_revenue, \
        'Number of Employees':employee_data}

df = pd.DataFrame(dict)
df

,Company Name,Industry,Headquarters Location,Estimated Annual Revenue,Number of Employees
0,Walmart,General Merchandisers,"Bentonville, Arkansas","$680,985","2,100,000"
1,Amazon,Internet Services and Retailing,"Seattle, Washington","$637,959","1,556,000"
2,UnitedHealth Group,Health Care: Insurance and Managed Care,"Eden Prairie, Minnesota","$400,278","400,000"
3,Apple,"Computers, Office Equipment","Cupertino, California","$391,035","164,000"
4,CVS Health,Health Care: Pharmacy and Other Services,"Woonsocket, Rhode Island","$372,809","259,500"
...,...,...,...,...,...
95,General Dynamics,Aerospace & Defense,"Reston, Virginia","$47,716","117,000"
96,Coca-Cola,Beverages,"Atlanta, Georgia","$47,061","69,700"
97,TIAA,"Insurance: Life, Health (Mutual)","New York, New York","$46,945.8","15,623"
98,Travelers,Insurance: Property and Casualty (Stock),"New York, New York","$46,423","34,000"


In [78]:
df['Company Name'][0]

'Walmart'

In [144]:
prompt = f"""
Given a company name, enrich its profile with public data on employee count, recent funding (last 12 months), key technologies, and AI/SaaS signals. Use web_search or browse_page on credible sources (e.g., Crunchbase, LinkedIn, company site, TechCrunch). Return a JSON object with:

- `company_name`: {df['Company Name'][65]}.
- `recent_funding`: Funding rounds from 09/25/2024 to 09/25/2025 (round type, amount, date MM/DD/YYYY, lead investor).Find the recent funding details if available. Bulleted list string or "No funding in last 12 months." or "No recent funding found. "
- `key_technologies`: 3-5 technologies/software used (e.g., "Python, AWS, Salesforce"). Comma-separated list.
- `ai_saas_signals`: 3-5 signals of AI/SaaS interest (e.g., press releases, job postings, news, last 12 months). Bulleted list string with source and date, or "No recent signals found."


Ethical: Use only public data, respect robots.txt. Limit tool calls to 5.

**Output Format** (JSON):
{{
  "company_name": "string",
  "recent_funding": "string",
  "key_technologies": "string",
  "ai_saas_signals": "string"
}}

"""

In [152]:
print('hi')
time.sleep(10)
print('hi')

hi
hi


In [145]:

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": prompt,
        }
    ],

    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0.1,
    response_format={"type": "json_object"}
)

print(chat_completion.choices[0].message.content)

{
  "company_name": "Allstate",
  "recent_funding": "No recent funding found.",
  "key_technologies": "Python, AWS, Salesforce, Java, Oracle",
  "ai_saas_signals": "• Allstate launches AI-powered insurance claims processing (TechCrunch, 02/15/2024)\n• Allstate job posting for Senior AI Engineer (LinkedIn, 08/20/2024)\n• Allstate partners with Google Cloud for AI-driven underwriting (BusinessWire, 05/10/2024)"
}


In [143]:
chat_completion

ChatCompletion(id='chatcmpl-23e9ab24-3d58-4127-b108-36b9b4b18d46', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "company_name": "Allstate",\n  "recent_funding": "No recent funding found. ",\n  "key_technologies": "Python, AWS, Salesforce, Java, Oracle",\n  "ai_saas_signals": "• Allstate launches AI-powered insurance claims processing (TechCrunch, 02/15/2024)\\n• Allstate job posting for AI/ML engineer (LinkedIn, 08/20/2024)\\n• Allstate partners with Google Cloud for AI-driven underwriting (BusinessWire, 05/10/2024)"\n}', role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1758794831, model='meta-llama/llama-4-maverick-17b-128e-instruct', object='chat.completion', system_fingerprint='fp_c527aa4474', usage=CompletionUsage(completion_tokens=122, prompt_tokens=303, total_tokens=425, completion_time=0.227896456, prompt_time=0.008105805, queue_time=0.209224986, total_time=0.236002

In [150]:
for company in df['Company Name']:
  data=[]
  prompt = f"""
Given a company name, enrich its profile with public data on employee count, recent funding (last 12 months), key technologies, and AI/SaaS signals. Use web_search or browse_page on credible sources (e.g., Crunchbase, LinkedIn, company site, TechCrunch). Return a JSON object with:

- `company_name`: {company}
- `recent_funding`: Funding rounds from 09/25/2024 to 09/25/2025 (round type, amount, date MM/DD/YYYY, lead investor).Find the recent funding details if available. Bulleted list string or "No funding in last 12 months." or "No recent funding found. "
- `key_technologies`: 3-5 technologies/software used (e.g., "Python, AWS, Salesforce"). Comma-separated list.
- `ai_saas_signals`: 3-5 signals of AI/SaaS interest (e.g., press releases, job postings, news, last 12 months). Bulleted list string with source and date, or "No recent signals found."


Ethical: Use only public data, respect robots.txt. Limit tool calls to 5.

**Output Format** (JSON):
{{
  "company_name": "string",
  "recent_funding": "string",
  "key_technologies": "string",
  "ai_saas_signals": "string"
}}

"""
  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": prompt,
        }
    ],

    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0.1,
    response_format={"type": "json_object"}
  )
  data.append(chat_completion.choices[0].message.content)


  print(chat_completion.choices[0].message.content)






hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
